# Week 3 Assignment-Part 1

In [2]:
import pandas as pd
import numpy as np

#### Read the table on html

In [3]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
ca_pc=df[0]
ca_pc.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Clean data

In [4]:
ca_pc = ca_pc.groupby(
    by=['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
ca_pc.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn


In [5]:
ca_pc.replace('Not assigned', np.nan, inplace=True)
ca_pc.dropna(subset=['Borough'], axis=0, inplace=True)
ca_pc.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [6]:
ca_pc.shape

(103, 3)

# Week 3 Assignment-Part 2

#### Merge the geoinfo table with the postcode table

In [7]:
!wget --quiet http://cocl.us/Geospatial_data -O lldata.csv

In [8]:
df=pd.read_csv('lldata.csv')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
geoinfo=ca_pc.merge(df, left_on='Postcode',right_on='Postal Code')
t_geoinfo=geoinfo[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
print(t_geoinfo.shape)
t_geoinfo.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Week 3 Assignment-Part 3

In [10]:
import folium
import pandas as pd
import numpy as np

#### Visualize boroughs on the folium map

In [11]:
# the location info of Toronto
latitude=43.700110
longitude=-79.416300

In [18]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(t_geoinfo['Latitude'], t_geoinfo['Longitude'], t_geoinfo['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

#### Define Foursquare credentials and versions

In [19]:
CLIENT_ID = '5S50NX55VPFLHVTLIBXW1JN1BWHX20HKZD5E3IBXX4PLFXVU' # your Foursquare ID
CLIENT_SECRET = 'YX2N01VTYZMWPDHH43H3SJJ5CCEKZM32XO2SFBOLO441FND0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5S50NX55VPFLHVTLIBXW1JN1BWHX20HKZD5E3IBXX4PLFXVU
CLIENT_SECRET:YX2N01VTYZMWPDHH43H3SJJ5CCEKZM32XO2SFBOLO441FND0


#### Fetch the information of activities around neighborhoods

In [20]:
import requests

In [21]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
tborough_venues=getNearbyVenues(names=t_geoinfo['Borough'], 
                                latitudes=t_geoinfo['Latitude'], 
                                longitudes=t_geoinfo['Longitude'], 
                                LIMIT=100,
                                radius=500)

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [23]:
print(tborough_venues.shape)
tborough_venues.head()

(2252, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [25]:
#save the dataframe
tborough_venues.to_csv('tborough_venues2.csv')

In [27]:
tborough_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,112,112,112,112,112,112
Downtown Toronto,1291,1291,1291,1291,1291,1291
East Toronto,126,126,126,126,126,126
East York,76,76,76,76,76,76
Etobicoke,78,78,78,78,78,78
Mississauga,11,11,11,11,11,11
North York,237,237,237,237,237,237
Queen's Park,38,38,38,38,38,38
Scarborough,83,83,83,83,83,83


### Analyze each neighborhood

#### Get dummies of each venues

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(tborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = tborough_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2252, 276)


,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Calculate the mean of venues of each borough

In [31]:
tborough_grouped=toronto_onehot.groupby('Borough').mean().reset_index()
print(tborough_grouped.shape)
tborough_grouped

(11, 276)


,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,...,0.000000,0.008929,0.000000,0.000000,0.008929,0.000000,0.000000,0.000000,0.000000,0.008929
1,Downtown Toronto,0.000000,0.000775,0.000775,0.000775,0.000775,0.001549,0.002324,0.001549,0.015492,...,0.000000,0.013168,0.002324,0.000000,0.004648,0.000000,0.006197,0.000775,0.000000,0.002324
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.013158
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.008439,0.000000,0.004219,0.000000,0.000000,0.000000,0.000000,0.000000,0.008439,...,0.000000,0.000000,0.004219,0.004219,0.008439,0.000000,0.000000,0.004219,0.016878,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.026316
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011236,0.000000,0.000000,0.011236,0.000000,0.005618,0.000000,0.000000,0.005618


#### Get the top 5 venues of each borough

In [32]:
num_top_venues = 5

for hood in tborough_grouped['Borough']:
    print("----"+hood+"----")
    temp = tborough_grouped[tborough_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.10
1  Sandwich Place  0.06
2            Park  0.05
3    Dessert Shop  0.04
4            Café  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.05
2           Restaurant  0.03
3                Hotel  0.03
4  Japanese Restaurant  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.03
4                Park  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.08
1          Pizza Place  0.05
2                 Park  0.05
3                 Bank  0.04
4  Sporting Goods Shop  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.10
1  Sandwich Place  0.06
2     Coffee Shop  0.06
3        Pharmacy  0.05
4             Gym  0.04


----Mississauga----
                       venue  freq
0                      Ho

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = tborough_grouped['Borough']

for ind in np.arange(tborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Pizza Place,Café,Sushi Restaurant,Dessert Shop,Clothing Store,Italian Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Gastropub,Steakhouse
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Ice Cream Shop,Brewery,Yoga Studio,Sandwich Place,Pub
3,East York,Coffee Shop,Pizza Place,Park,Pharmacy,Burger Joint,Sporting Goods Shop,Bank,Gym,Liquor Store,Pet Store
4,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Gym,Café,Fast Food Restaurant,Grocery Store,Discount Store,Fried Chicken Joint


#### Visualize clusters

In [43]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

tborough_grouped_clustering = tborough_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 3, 0, 1, 4, 3, 1], dtype=int32)

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tborough_merged = t_geoinfo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tborough_merged = tborough_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

print(tborough_merged.shape)
tborough_merged.head() # check the last columns!

(103, 16)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink


#### Cluster boroughs

In [45]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

In [46]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [50]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tborough_merged['Latitude'], tborough_merged['Longitude'], tborough_merged['Borough'], tborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

In [51]:
tborough_merged.loc[tborough_merged['Cluster Labels'] == 0, tborough_merged.columns[[1] + list(range(5, tborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Mississauga,0,Hotel,Coffee Shop,Burrito Place,Gym / Fitness Center,Fried Chicken Joint,Sandwich Place,Mediterranean Restaurant,American Restaurant,Middle Eastern Restaurant,Dog Run


In [52]:
tborough_merged.loc[tborough_merged['Cluster Labels'] == 1, tborough_merged.columns[[1] + list(range(5, tborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
18,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
19,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
20,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
21,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
22,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
23,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
24,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
25,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place
26,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Grocery Store,Park,Pizza Place,Bank,Restaurant,Sandwich Place


In [53]:
tborough_merged.loc[tborough_merged['Cluster Labels'] == 2, tborough_merged.columns[[1] + list(range(5, tborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,York,2,Park,Convenience Store,Pharmacy,Bar,Women's Store,Hockey Arena,Field,Fast Food Restaurant,Market,Discount Store
74,York,2,Park,Convenience Store,Pharmacy,Bar,Women's Store,Hockey Arena,Field,Fast Food Restaurant,Market,Discount Store
80,York,2,Park,Convenience Store,Pharmacy,Bar,Women's Store,Hockey Arena,Field,Fast Food Restaurant,Market,Discount Store
81,York,2,Park,Convenience Store,Pharmacy,Bar,Women's Store,Hockey Arena,Field,Fast Food Restaurant,Market,Discount Store
98,York,2,Park,Convenience Store,Pharmacy,Bar,Women's Store,Hockey Arena,Field,Fast Food Restaurant,Market,Discount Store


In [54]:
tborough_merged.loc[tborough_merged['Cluster Labels'] == 3, tborough_merged.columns[[1] + list(range(5, tborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
1,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
2,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
3,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
4,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
5,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
6,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
7,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
8,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink
9,Scarborough,3,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bus Line,Intersection,Skating Rink


In [55]:
tborough_merged.loc[tborough_merged['Cluster Labels'] == 4, tborough_merged.columns[[1] + list(range(5, tborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Queen's Park,4,Coffee Shop,Gym,Park,Smoothie Shop,Seafood Restaurant,Burger Joint,Burrito Place,Sandwich Place,Café,Chinese Restaurant
